In [1]:
'''
1. 将拼多多收入与订单合并
2. 对收入分别按订单汇总、按存货编码汇总
202405-202409
保留中间过程
'''

import easygui
import os
import pandas as pd
import re
import numpy as np



In [2]:
regax = r'（订单(?P<dingdanhao>\d{6}-\d{15})）'
pattern = re.compile(regax)
# mat = pattern.search(string)
def getDingdanhao(s):
    mat = pattern.search(s)
    if mat :
        s = mat.group('dingdanhao')
    else :
        s = ''
    return s

#所有收入
def chuliFnameShouru(fname):
    # fname_shouru = r"F:\a00nutstore\008\zw08\电商\拼多多\pdd-mall-bill-detail(100306881)_2024-09-30-07-09-49_3.csv"
    df_shouru0 = pd.read_csv(fname,header = 4,encoding = 'gb18030')
    df_shouru1 = df_shouru0[['商户订单号','发生时间','收入金额（+元）','支出金额（-元）','账务类型','备注','业务描述']]
    df_shouru1 = df_shouru1.dropna(thresh = 2)   #至少有三个缺失值 
    df_shouru1['商户订单号']= df_shouru1['商户订单号'].mask(df_shouru1['商户订单号'].isnull(),df_shouru1['备注'].map(getDingdanhao))
    df_shouru1['商户订单号'] = df_shouru1['商户订单号'].replace('','888888-888888888888888')
    df_shouru1.columns = ['dingdanhao', 'riqi', 'ru', 'chu', 'leixing', 'beizhu', 'miaoshu']
    df_shouru2 = df_shouru1.loc[df_shouru1.leixing != '提现']
    df_shouru2 = df_shouru2.assign(shouru = df_shouru2['ru'] + df_shouru2['chu'])
    return df_shouru2

dic = {'五': 5,
       '六': 6,
       '七': 7,
       '八': 8,
       '九': 9,
       '十': 10,
       '四': 4,
       '三': 3,
       '二': 2,
       '一': 1}


def addBen(string):
    # string = r'(81本)'
    pattern1 = r'(?P<ben>\d+)本'
    pattern2 = r'(?P<ben>[十九八七六五四三二一])本'
    regexp1 = re.compile(pattern1)
    mat1 = regexp1.search(string)
    regexp2 = re.compile(pattern2)
    mat2 = regexp2.search(string)
    if mat1:
        ben = int(mat1.group('ben'))
    else:
        if mat2:
            ben = dic.get(mat2.group('ben'))
        else:
            ben = 1
    return ben



In [8]:
#所有订单
path_dingdan = r'F:\a00nutstore\008\zw08\电商\拼多多\拼多多订单创业至20240930'
data_dingdan = []
for i in os.listdir(path_dingdan):
    j = os.path.join(path_dingdan,i)
    if  os.path.isfile(j) :
        df = pd.read_csv(j)
        data_dingdan.append(df)
    else :
        continue




def readDingdan(fname) :
    df_dingdan = pd.read_csv(fname)
    return df_dingdan

def chuliDingdan(df_dingdan0):
    df_dingdan1 = df_dingdan0[['订单号','发货时间','商家编码-规格维度','商家实收金额(元)','商品数量(件)','商品规格']]
    df_dingdan1.columns = ['dingdanhao','riqi','code','jiner','bao','guige']
    df_dingdan1[df_dingdan1['dingdanhao'].isnull()]
    df_dingdan1['hanliang'] = df_dingdan1['guige'].map(addBen)
    df_dingdan1 = df_dingdan1.assign(ben=df_dingdan1['bao'] * df_dingdan1['hanliang'])
    print(df_dingdan1.columns.to_list())
    df_dingdan2 = df_dingdan1['dingdanhao','riqi','code','jiner','bao','guige','ben']
    df_dingdan2 = df_dingdan2.set_index('dingdanhao')
    return df_dingdan2


df_dingdan0 = pd.concat(data_dingdan)
df_dingdan2 = chuliDingdan(df_dingdan0)
        
df_dingdan2.head()


['dingdanhao', 'riqi', 'code', 'jiner', 'bao', 'guige', 'hanliang', 'ben']


C:\Users\redda\AppData\Local\Temp\ipykernel_20628\4232253340.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dingdan1['hanliang'] = df_dingdan1['guige'].map(addBen)


KeyError: ('dingdanhao', 'riqi', 'code', 'jiner', 'bao', 'guige', 'ben')

In [9]:
df_dingdan0.head()

,商品,订单号,订单状态,商品数量(件),是否审核中,支付时间,承诺发货时间,承诺送达时间,发货时间,确认收货时间,...,预约配送时间,是否节能补贴,是否分期,分期期数,手续费承担方,是否无痕发货,订单来源,是否社区团购,小区送达情况,分期方式
0,500页加厚英语定位练习纸本英语定位纸英文专用纸小学初中高中,240929-658935703962749,已收货,1,正常,2024-09-29 23:49:15,2024-10-01 23:49:15,NaN,2024-09-30 11:14:43,2024-10-07 22:43:37,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN
1,500页加厚英语定位练习纸本英语定位纸英文专用纸小学初中高中,240929-379762829722749,已收货,1,正常,2024-09-29 23:49:15,2024-10-01 23:49:15,NaN,2024-09-30 11:14:43,2024-10-07 22:43:44,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN
2,莱特5本加厚200页B5学生笔记本办公记事本白领工作记录本子笔记本,240929-211738969641353,已收货,1,正常,2024-09-29 23:44:20,2024-10-01 23:44:20,NaN,2024-09-30 11:14:43,2024-10-06 23:51:52,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN
3,莱特5本加厚200页B5学生笔记本办公记事本白领工作记录本子笔记本,240929-611277893212149,已发货，待收货,1,正常,2024-09-29 23:37:17,2024-10-01 23:37:18,NaN,2024-09-30 11:14:43,\t,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN
4,莱特5本加厚200页B5学生笔记本办公记事本白领工作记录本子笔记本,240929-080425849263824,未发货，退款成功,1,正常,2024-09-29 23:34:51,2024-10-01 23:34:51,NaN,\t,\t,...,\t,否,否,NaN,\t,否,\t,否,NaN,NaN


In [4]:
#拼多多收入
path = r'F:\a00nutstore\008\zw08\电商\拼多多\拼多多各月收入'
# os.chdir(path)
#莱新销售订单超5000条，不能一次导出，分三次导出，并分别存于同一文件下，先将它们合并
data = []
for i in os.listdir(path):
    j = os.path.join(path,i)
    if  os.path.isfile(j):
        df = chuliFnameShouru(j)
        data.append(df)
    else :
        continue


df_xiaoshou0 = pd.concat(data)
df_xiaoshou0.head()

,dingdanhao,riqi,ru,chu,leixing,beizhu,miaoshu,shouru
0,240529-670721687811391,2024/5/31 23:35,0.00,-0.05,技术服务费,先用后付技术服务费,0030002|技术服务费-基础技术服务费,-0.05
1,240529-670721687811391,2024/5/31 23:35,0.45,0.00,优惠券结算,交易成功优惠券支付金额结算,0010005|交易收入-优惠券结算,0.45
2,240529-670721687811391,2024/5/31 23:35,4.05,0.00,交易收入,-,0010002|交易收入-订单收入,4.05
3,240527-100804923722069,2024/5/31 22:45,7.80,0.00,交易收入,-,0010002|交易收入-订单收入,7.80
4,240527-100804923722069,2024/5/31 22:45,0.00,-7.80,退款,-,0020002|交易退款-订单退款,-7.80


In [5]:
shouru_pivot =df_xiaoshou0.pivot_table(index = 'dingdanhao',values =['ru','chu','shouru'],aggfunc = np.sum)
result0 =pd.merge(shouru_pivot,df_dingdan2,left_index = True,right_index = True,how = 'left')
#空值填充为0
for i in ['chu','ru','shouru','jiner','shuliang']:
    result0[i] = result0[i].fillna(0)
#code填充
result0['code'] = result0['code'].fillna('8888')

#日期填充
result0['riqi'] = result0['riqi'].replace('\t',None)
result0['riqi'] = result0['riqi'].ffill()
result0['riqi'] = result0['riqi'].bfill()
result0.head()

C:\Users\redda\AppData\Local\Temp\ipykernel_23052\1342555680.py:1: FutureWarning: The provided callable <function sum at 0x000001B6DA662C00> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  shouru_pivot =df_xiaoshou0.pivot_table(index = 'dingdanhao',values =['ru','chu','shouru'],aggfunc = np.sum)


,chu,ru,shouru,riqi,code,jiner,shuliang
dingdanhao,,,,,,,
240508-045398148751866,-0.03,4.80,4.77,2024-05-09 10:24:39,\t,4.80,3.0
240508-045618327280511,-0.04,6.30,6.26,2024-05-09 10:24:14,\t,6.30,5.0
240508-070988623551931,-0.06,10.50,10.44,2024-05-09 10:24:39,\t,10.50,10.0
240508-185692363713721,-0.06,9.89,9.83,2024-05-09 10:24:39,DWXB5,9.89,5.0
240508-188271860572818,-0.06,10.50,10.44,2024-05-09 10:24:39,\t,10.50,10.0


In [6]:
gp = result0.groupby('dingdanhao')
#将订单总金额按各产品的明细销售额进行分配
data = []
for k,v in gp:
    # print(k,v)
    mean = v.shouru.mean()
    total = v['jiner'].sum()
    # print('mean',mean)
    # print('total',total)
    v['result_jiner'] = v.shouru*(v.jiner)/total
    data.append(v)
result1 = pd.concat(data)
result1 = result1.reset_index()


In [7]:
result1 = result1.rename({'riqi': '日期',
 'dingdanhao': '订单号',
 'ru': '收入',
 'chu': '支出',
 'shouru': '净收入',
 'code': '存货编码',
 'jiner': '订单金额',
 'shuliang': '数量',
 'result_jiner': '最终金额'},axis = 1)
result1  = result1[['日期', '订单号', '收入', '支出', '净收入', '存货编码', '订单金额', '数量', '最终金额']]
result1 ['最终金额']= np.where(result1 ['最终金额'].isnull(),result1 ['净收入'],result1['最终金额'])
# result1.to_excel('分配后15.xlsx',index=False)

In [8]:
total = result1.sum().T.to_frame().T
total.日期  = '合计'
total.订单号  = ''
total.存货编码  = ''
result2 = pd.concat([result1,total])
result2.to_excel('拼多多202405-202409净收入明细.xlsx',index = False)

In [ ]:
# result1 = result1.rename({'':'888888-888888888888888'})
# result0.to_excel('分配前outer-2-8.xlsx')
# df_dingdan2.to_excel('订单号-8.xlsx')


In [ ]:
df00 = pd.read_excel(r"F:\repos\fisha\拼多多202409净收入明细.xlsx")
df00.columns.to_list()

In [ ]:
result1.columns.to_list()

In [ ]:
dic = dict(zip(['riqi','dingdanhao','ru','chu',  'shouru',  'code', 'jiner', 'shuliang', 'result_jiner'],
['日期', '订单号', '收入', '支出', '净收入', '存货编码', '订单金额', '数量', '最终金额']))
dic

In [ ]:
df11 = pd.read_clipboard()
df11

In [ ]:
import numpy as np
df11['最终金额']= np.where(any(df11['最终金额'].isnull()*df11['最终金额'] == '' == 0),df11['收入'],df11['最终金额'])
df11['最终金额']

In [ ]:
df11

In [ ]:
df11

In [ ]:
df22 = df11.copy()
df22

In [ ]:
df22['最终金额']= np.where(df22['最终金额'].isnull(),df22['净收入'],df22['最终金额'])
df22

In [ ]:
df22['最终金额'].isnull()

In [ ]:
df23 = df22.sum().T.to_frame().T
df23.日期  = '合计'
df23.订单号  = ''
df23.存货编码  = ''
df23

In [ ]:
df24 = df22.sum().str.replace('日期','')

In [ ]:
type(df22.sum())

In [ ]:
df24
